In [ ]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
from re import sub

df = pd.read_csv('[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv', skiprows=1)
df = df.iloc[:, :9]
df_ocr = pd.read_csv('20220209190321A17.csv', skiprows=1)

In [ ]:
df_ocr.columns = ['no', 'id', 'fullname', 'party', 'vote']
df_ocr = df_ocr.astype(np.str_)

In [ ]:
def similar(a, b):
    return SequenceMatcher(lambda x: x in ' ', a, b).ratio()

In [ ]:
def fix_party(p, ps):
  if not p: return p

  for x in ps:
    sim = similar(p, str(x))
    if sim > .8:
      return x
  return p

df_ocr.party = df_ocr.party.apply(lambda x: fix_party(x.replace('พรรศ', 'พรรค').replace('พรวศ', 'พรรค').replace('พรรค', ''), df.party.unique()))

In [ ]:
party_b = df_ocr.party.isin(df.party.dropna())

In [ ]:
TITLE = ['ร้อยตํารวจตรี', 'พันเอก', 'ร้อยตํารวจเอก', 'พันตํารวจโท', 'พลตํารวจเอก', 'พันตํารวจเอก']
REPLACE = ('นนางสาว', 'นางสาว'), ('นษาง', 'นาง'), ('นบนาย', 'นาย'), ('บาย', 'นาย'), ('พสตํารวจ', 'พลตำรวจ'), ('พลตำรวจตรวี', 'พลตำรวจตรี')

def split_name(name, titles):
  name = sub('[!-~]', '', name)
  for o, n in REPLACE:
    if name.startswith(o):
      name = name.replace(o, n)

  name = sub('(ร้อย|พัน|พล)(ตํารวจ)?(ตรี|โท|เอก)', '', name).strip()

  for t in titles:
    if name.startswith(t):
      return name[len(t):]
  return name

df_ocr.fullname = df_ocr.fullname.apply(lambda x: split_name(x, df.title.unique().tolist()+TITLE))

In [ ]:
def vote_encoder(vote: str):
  '''
  1 = เห็นด้วย,
  2 = ไม่เห็นด้วย,
  3 = งดออกเสียง,
  4 = ไม่ลงคะแนนเสียง, 
  5 = ไม่เข้าร่วมประชุม,
  \- = ไม่ใช่วาระการประชุม
  '''
  if vote == 'เห็นด้วย':
    return 1
  if vote == 'ไม่เห็นด้วย':
    return 2
  if vote == 'งดออกเสียง':
    return 3
  if vote == 'ไม่ลงคะแนนเสียง':
    return 4
  if vote == '-':
    return 5

  p = (similar(vote,'เห็นด้วย'), similar(vote,'ไม่เห็นด้วย'), similar(vote,'งดออกเสียง'), similar(vote, 'ไม่ลงคะแนนเสียง'))
  i = np.argmax(p)
  if p[i] < .8:
    return None
  return i + 1

In [ ]:
df_ocr.vote = df_ocr.vote.apply(vote_encoder)

In [ ]:
name_b = df_ocr.fullname.apply(lambda x: x.split(' ')[0]).apply(lambda x: (x == df.name.dropna()).sum())

In [ ]:
def last_name(fullname):
  sep_fn = fullname.split(' ', maxsplit=1)
  if (len(sep_fn) > 1):
    return sep_fn[-1].replace('\'', '')
  return fullname

last_name_b = df_ocr.fullname.apply(last_name).isin(df.lastname.dropna())

In [ ]:
name_b.sum(),last_name_b.sum(),(name_b|last_name_b).sum(),party_b.sum()

In [ ]:
included_b = (name_b)|(last_name_b)|(party_b)
df_cleaned = df_ocr[included_b]
df_cleaned

In [ ]:
match_row = np.full(df.name.shape, np.nan)

for i, g in df.groupby('party'):
  tmp = df_cleaned[df_cleaned['party'] == i]
  for j, r in tmp.iterrows():
    sim = g.apply(lambda x: similar(r.fullname.replace(' ', ''), x['name'] + x['lastname']), axis=1)
    if sim.max() > .6:
      if j not in match_row:
        match_row[g.iloc[sim.argmax()].name] = j

for i, row in df_ocr[included_b&(df_ocr.index.isin(np.unique(match_row))==False)].iterrows():
  sim = df.apply(lambda x: similar(row.fullname.replace(' ', ''), x['name'] + x['lastname']), axis=1)
  if sim.max() > .7:
    match_row[sim.argmax()] = i

df_ocr[included_b&(df_ocr.index.isin(np.unique(match_row))==False)]

In [ ]:
df['ocr'] = match_row

In [ ]:
merged_df = df.merge(df_ocr[['fullname', 'vote']], left_on='ocr', right_index=True)
merged_df

In [ ]:
df_ch = pd.read_csv('[WeVis] They Work for Us - Politician Data - [T] PeopleVote.csv', skiprows=1)

In [ ]:
len(df_ch['votelog.__82']),len(vote_82_ocr)

In [ ]:
vote_82_ocr[(df_ch['votelog.__82'].values==vote_82_ocr.apply(str)).values]

In [ ]:
df_ocr.loc[match_row].vote[(df_ch['votelog.__82'].replace('-', 5).astype(np.float16).values==vote_82_ocr.values)==False]

In [ ]:
df_ch['votelog.__82'].isna().sum(),vote_82_ocr.isna().sum()

In [ ]:
(df_ocr.loc[match_row])[df_ch['votelog.__82']=='1'].sample(10)